## ***DIPLOMADO EN CIENCIA DE DATOS APLICADOS AL DESARROLLO INTEGRAL DEL TALENTO HUMANOX***  

### ***M4: Análisis Predictivo para la Toma de Decisiones Estratégicas en RRHH***  

### ***<span style="color:#FF0000">Universidad del Rosario</span>***

---

# 📘 Notebook 3 – Comparación y Optimización de Modelos de Regresión

En este notebook avanzaremos un paso más en el análisis predictivo aplicado a **Recursos Humanos**.  
El objetivo principal será **comparar distintos modelos de regresión** para predecir una variable númerica y luego **mejorar el desempeño del modelo más prometedor**.

## 🎯 Objetivos
1. **Comparar múltiples algoritmos de regresión** (Lineal, Ridge, Lasso, Árboles de decisión, Random Forest, entre otros).  
2. **Seleccionar el mejor modelo** con base en métricas de desempeño como MAE, RMSE y R².  
3. **Optimizar el modelo elegido** mediante técnicas de ajuste de hiperparámetros (*GridSearchCV* o *RandomizedSearchCV*).  
4. **Visualizar y analizar los resultados**, identificando patrones y oportunidades de mejora.  

## 🛠️ Herramientas
- **scikit-learn** → construcción y evaluación de modelos.  
- **pandas / numpy** → manipulación de datos.  
- **matplotlib / seaborn** → visualización de resultados.  

➡️ Al finalizar este notebook, contaremos con un modelo optimizado y validado que servirá como punto de referencia sólido para la predicción de competencias blandas en el contexto de RRHH.


In [ ]:
# Instalación de librerías necesarias
# 
# En esta celda instalamos las principales librerías que usaremos a lo largo del notebook:
# - pandas: para manipulación y análisis de datos.
# - numpy: para operaciones numéricas y manejo de arreglos.
# - matplotlib: para visualizaciones básicas.
# - seaborn: para visualizaciones estadísticas más estilizadas.
# - scikit-learn: para construir y evaluar modelos de machine learning.
#
# Esta celda solo necesita ejecutarse una vez (o cuando el entorno no tenga estas librerías instaladas).

!pip install pandas numpy matplotlib seaborn scikit-learn

In [ ]:
# Librerías necesarias

# 🔹 Manipulación y análisis de datos
import pandas as pd
import numpy as np

# 🔹 Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# 🔹 Preprocesamiento y modelado
from sklearn.model_selection import train_test_split   # División en train y test
from sklearn.linear_model import LinearRegression      # Modelo de regresión lineal

# 🔹 Evaluación del modelo
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
# 📥 Cargar dataset
url = "https://raw.githubusercontent.com/LeStark/Cursos/refs/heads/main/Data/iris_human_resources.csv"
df = pd.read_csv(url, sep=",")
df.info()  # Información del dataset
df.head()  # Mostrar las primeras filas del dataset